# Testing Custom Timeouts


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core```)


# Create Cluster

Start minikube and ensure custom resource validation is activated and there is 5G of memory. 

Your start command with the kvm driver would then look like:
```
minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC
```

# Setup

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


# Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/janis/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [5]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

Install the custom resource definition

In [6]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Tue Dec 18 11:10:48 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s


NOTES:
NOTES: TODO




In [7]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon \
    --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Tue Dec 18 11:10:51 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Service
NAME                          TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador-admin  NodePort   10.108.119.46   <none>       8877:30383/TCP                 1s
seldon-core-ambassador        NodePort   10.110.98.62    <none>       80:31892/TCP,443:32336/TCP     1s
seldon-core-seldon-apiserver  NodePort   10.105.205.239  <none>       8080:30495/TCP,5000:30228/TCP  0s
seldon-core-redis             ClusterIP  10.107.101.105  <none>       6379/TCP                       0s

==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-ambassador              1        1        1           0          0s
seldon-core-seldon-apiserver        1        1        1           0          0s
seldon-core-seldon-cluster-manager  1        1        1           0          0s
seldon-core-

Check all services are running before proceeding.

In [8]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador 

Waiting for deployment "seldon-core-seldon-cluster-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
Waiting for deployment "seldon-core-seldon-apiserver" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward to API Gateway**

REST:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

GRPC:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:5000
```

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8005:8080
```

Illustration of both REST and gRPC requests. 

In [9]:
import requests
from requests.auth import HTTPBasicAuth
from seldon_core.proto import prediction_pb2
from seldon_core.proto import prediction_pb2_grpc
import grpc
import numpy as np

AMBASSADOR_API="localhost:8005"
GATEWAY_REST="localhost:8003"
GATEWAY_GRPC="localhost:8004"

def get_token(oauth_key,oauth_secret):
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://"+GATEWAY_REST+"/oauth/token",
                auth=HTTPBasicAuth(oauth_key, oauth_secret),
                data=payload)
    print(response.text)
    token =  response.json()["access_token"]
    return token

    
def rest_request_api_gateway(oauth_key,oauth_secret):
    token = get_token(oauth_key,oauth_secret)
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+GATEWAY_REST+"/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print(response.text)

def grpc_request_api_gateway(oauth_key,oauth_secret,data_size):
    token = get_token(oauth_key,oauth_secret)
    shape = [1,data_size]
    arr = np.random.rand(data_size)
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = shape,
                values = arr
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(GATEWAY_GRPC)
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print(response)
    
def grpc_request_ambassador(deploymentName,data_size):
    shape = [1,data_size]
    arr = np.random.rand(data_size)
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = shape,
                values = arr
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(AMBASSADOR_API)
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('seldon',deploymentName)]
    response = stub.Predict(request=request,metadata=metadata)
    print(response)

In [10]:
!pygmentize resources/model_short_timeouts.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction",
            "deployment_version": "v1",
	    "seldon.io/rest-read-timeout":"1"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                          

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [11]:
!kubectl apply -f resources/model_short_timeouts.json -n seldon

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [12]:
!kubectl get seldondeployments -n seldon

NAME                        CREATED AT
seldon-deployment-example   2s


In [13]:
!kubectl describe seldondeployments seldon-deployment-example -n seldon

Name:         seldon-deployment-example
Namespace:    seldon
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration:
                {"apiVersion":"machinelearning.seldon.io/v1alpha2","kind":"SeldonDeployment","metadata":{"name":"seldon-deployment-example","namespace":"s...
API Version:  machinelearning.seldon.io/v1alpha2
Kind:         SeldonDeployment
Metadata:
  Cluster Name:        
  Creation Timestamp:  2018-12-18T11:12:01Z
  Generation:          1
  Resource Version:    746
  Self Link:           /apis/machinelearning.seldon.io/v1alpha2/namespaces/seldon/seldondeployments/seldon-deployment-example
  UID:                 be7a2bce-02b5-11e9-aed1-18941d3b439b
Spec:
  Annotations:
    Deployment Version:                   v1
    Project Name:                         FX Market Prediction
    Seldon . Io / Rest - Read - Timeout:  1
  Name:                                   test-deployment
  Oauth Key:                              oauth-key
  Oauth Secre

Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [15]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[replicasAvailable:1 name:test-deployment-fx-market-predictor-7cd068f replicas:1]] state:Available]

## Get predictions with short and long timeouts

This next request should fail as the timeout is too short

In [16]:
rest_request_api_gateway('oauth-key','oauth-secret')

{"access_token":"7bdc8db7-e6a9-4a4a-80c9-c208992bff1b","token_type":"bearer","expires_in":43199,"scope":"read write"}
{
  "code": 103,
  "info": "Status code: 500 Reason: ",
  "reason": "Microservice error",
  "status": "FAILURE"
}


Delete this graph and recreate one with a longer timeout

In [17]:
!kubectl delete -f resources/model_short_timeouts.json

seldondeployment.machinelearning.seldon.io "seldon-deployment-example" deleted


In [18]:
!pygmentize resources/model_long_timeouts.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction",
            "deployment_version": "v1",
	    "seldon.io/rest-read-timeout":"10000",
	    "seldon.io/rest-connection-timeout":"10000",	    
	    "seldon.io/grpc-read-timeout":"10000"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                    

In [19]:
!kubectl apply -f resources/model_long_timeouts.json -n seldon

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [21]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n seldon

map[state:Available predictorStatus:[map[name:test-deployment-fx-market-predictor-7cd068f replicas:1 replicasAvailable:1]]]

This next request should work as the timeout is much longer

In [22]:
rest_request_api_gateway('oauth-key','oauth-secret')

{"access_token":"7bdc8db7-e6a9-4a4a-80c9-c208992bff1b","token_type":"bearer","expires_in":43126,"scope":"read write"}
{
  "meta": {
    "puid": "8jq85h9pbdquf9uii6nh6majr7",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}
